In [1]:
!pip install -U segmentation-models

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 50 kB 2.6 MB/s 


In [2]:
import os
import numpy as np
import pandas as pd
from skimage import color
import matplotlib.pyplot as plt
from IPython.display import clear_output
import tensorflow as tf
import keras
import segmentation_models as sm
import numpy.ma as ma
import torch
from segmentation_models.utils import set_trainable

Segmentation Models: using `keras` framework.


In [3]:
# Turning on GPU to load data
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

device = tf.device(device_name)

Found GPU at: /device:GPU:0


In [4]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


# Data Clean Up Functions

In [5]:
'''
1. split into input and label
2. considered flair, t1, t1c, t2 as multiple inputs
3. repeated the label 4 times to match the inputs
4. considered each of the frames as inputs
'''
def process_samples(data):
  data = np.swapaxes(data, 2, 3)
  data = np.swapaxes(data, 1, 2)
  X = data[:4,:,:,:]
  y = data[4,:,:,:]
  y = np.reshape(y, (1, -1, 240, 240))
  y = np.repeat(y, 4, axis=0)
  return X, y

In [6]:
def three_channels(X, y):
  X = np.stack((X,)*3, axis=-1)
  y = np.stack((y,)*3, axis=-1)
  return X, y

In [7]:
def normalize_data(X):
  X = X/np.max(X)
  return X

In [8]:
def split_labels(label):
    label = label[:, :, :, :, 0] 
    label_ret = np.empty((label.shape[0], label.shape[1], label.shape[2],  label.shape[3], 0))
    for i in range(5):
      if i != 3:
        label_add = np.zeros(label.shape)
        label_add[label == i] = 1
        label_add = label_add[:,:,:,:, np.newaxis]
        label_ret = np.concatenate((label_ret, label_add), axis = 4)
    return label_ret

In [9]:
base_path = "/content/drive/MyDrive/Bioimaging/Bioimagining Project/data"
'''
batch_name : [0,50]
type: HGG or LGG or mix
split: train, val, test

Returns:
X and y 
'''
def load_data(batch_num, type_GG, split):
    file_name = "{}_{}".format(type_GG, split)
    X_file = "{}_X_{}_{}.npy".format(split, type_GG, batch_num)
    y_file = "{}_y_{}_{}.npy".format(split, type_GG, batch_num)
    X_path = os.path.join(base_path, "Unet_data", file_name, X_file)
    y_path = os.path.join(base_path, "Unet_data", file_name, y_file)

    return np.load(X_path), np.load(y_path)

In [10]:
def process_data_more(batch_num, type_GG, split):
  #load and reshape
  X, y = load_data(batch_num, type_GG, split)
  # X = np.reshape(X, (-1, X.shape[2], X.shape[3], X.shape[4]))
  # y = np.reshape(y, (-1, y.shape[2], y.shape[3], y.shape[4]))

  #pad to fit model input
  X = np.pad(X, ((0,0), (0,0), (16, 0), (16, 0), (0,0)), mode = 'constant')
  y = np.pad(y, ((0,0), (0,0), (16, 0), (16, 0), (0,0)), mode = 'constant')

  # split label into classes
  y = split_labels(y)

  nb_mask = np.sum(y[:, :, :, :, 1:], axis = (0, 2, 3, 4)) > 0
  X = X[:, nb_mask, :, :, :]
  y = y[:, nb_mask, :, :, :]

  #convert to tensors
  X = tf.convert_to_tensor(X, dtype=tf.float64)
  y = tf.convert_to_tensor(y, dtype=tf.float32)

  return X, y

In [11]:
# train_X, train_y = process_data_more(0, "mix", "train")
# print(train_y.shape, train_X.shape)


# Load In Data

In [12]:
def initialize_unet(backbone):
  preprocess_input = sm.get_preprocessing(backbone)
  optim = keras.optimizers.Adam(0.0001)
  # define model
  model = sm.Unet(backbone, encoder_weights='imagenet', classes=4, encoder_freeze=True, activation="sigmoid")
  model.compile(
      optim,
      loss=sm.losses.DiceLoss(),
      metrics=[sm.metrics.IOUScore(), sm.metrics.FScore()],
  )
  return model

In [ ]:
initialize_unet('resnet152').summary()

In [ ]:
train_X, train_y, val_X, val_y = None, None, None, None

In [13]:
optim = keras.optimizers.Adam(0.0001)
num_split = 50
model_name = "inceptionv3"
model_path = "/content/drive/MyDrive/Bioimaging/Bioimagining Project/UNet_model/testing_backbones"
model_path = os.path.join(model_path, model_name)

#metrics
# loss_lst = []
# val_loss_lst = []
# iou_lst = []
# val_iou_lst = []
# f1_lst = []
# val_f1_lst = []

for i in range(28, num_split):
  print(i)
  train_X, train_y = process_data_more(i, "mix", "train")
  val_X, val_y = process_data_more(i, "mix", "val")
  # loading just flair
  # train_X, train_y = train_X[1, :, :, :, :], train_y[1, :, :, :, :]
  # val_X, val_y = val_X[1, :, :, :, :], val_y[1, :, :, :, :]

  # loading in all image modalities
  train_X, train_y = np.reshape(train_X, (-1, train_X.shape[2], train_X.shape[3], train_X.shape[4])) , np.reshape(train_y, (-1, train_y.shape[2], train_y.shape[3], train_y.shape[4]))
  val_X, val_y = np.reshape(val_X, (-1, val_X.shape[2], val_X.shape[3], val_X.shape[4])) , np.reshape(val_y, (-1, val_y.shape[2], val_y.shape[3], val_y.shape[4]))
  # test_X, test_y = process_data_more(i, "mix", "test")

  # print(val_y.shape)
  # print(np.sum(val_y[:, :, :, 3]))

  if os.path.exists(model_path):
    model = keras.models.load_model(model_path, compile = False)
    model.compile(
      optim,
      loss=sm.losses.DiceLoss(),
      metrics=[sm.metrics.IOUScore(), sm.metrics.FScore()],
    )
  else:
    BACKBONE = 'inceptionv3'
    model = initialize_unet(BACKBONE)
  
  # if num_split < 3:
  #   history = model.fit(
  #   x=train_X,
  #   y=train_y,
  #   batch_size=16,
  #   epochs=3,
  #   validation_data=(val_X, val_y))
    
  #   model.save(model_path)
  #   train_X, train_y, val_X, val_y = None, None, None, None
  # else:
  #   set_trainable(model, recompile=False)

  #   history = model.fit(
  #   x=train_X,
  #   y=train_y,
  #   batch_size=16,
  #   epochs=3,
  #   validation_data=(val_X, val_y))

  set_trainable(model, recompile=False)

  history = model.fit(
  x=train_X,
  y=train_y,
  batch_size=16,
  epochs=3,
  validation_data=(val_X, val_y))

  # save metrics
  # loss_lst.append(history.history['loss'])
  # val_loss_lst.append(history.history['val_loss'])
  # iou_lst.append(history.history['iou_score'])
  # val_iou_lst.append(history.history['val_iou_score'])
  # f1_lst.append(history.history['f1-score'])
  # val_f1_lst.append(history.history['val_f1-score'])
  
  model.save(model_path)
  train_X, train_y, val_X, val_y = None, None, None, None

  # print(f'loss = {loss_lst}')
  # print(f'val_loss = {val_loss_lst}')
  # print(f'iou = {iou_lst}')
  # print(f'val_iou = {val_iou_lst}')
  # print(f'f1 = {f1_lst}')
  # print(f'val_f1 = {val_f1_lst}')

28


Epoch 1/3
67/67 [==============================] - 54s 518ms/step - loss: 0.2599 - iou_score: 0.6248 - f1-score: 0.7403 - val_loss: 0.5342 - val_iou_score: 0.4281 - val_f1-score: 0.4669
Epoch 2/3
67/67 [==============================] - 27s 404ms/step - loss: 0.2098 - iou_score: 0.6794 - f1-score: 0.7899 - val_loss: 0.5288 - val_iou_score: 0.4330 - val_f1-score: 0.4724
Epoch 3/3
67/67 [==============================] - 27s 397ms/step - loss: 0.2009 - iou_score: 0.6902 - f1-score: 0.7988 - val_loss: 0.5261 - val_iou_score: 0.4359 - val_f1-score: 0.4750


29


Epoch 1/3
57/57 [==============================] - 30s 383ms/step - loss: 0.3101 - iou_score: 0.5689 - f1-score: 0.6902 - val_loss: 0.6013 - val_iou_score: 0.3501 - val_f1-score: 0.3958
Epoch 2/3
57/57 [==============================] - 21s 360ms/step - loss: 0.2604 - iou_score: 0.6199 - f1-score: 0.7397 - val_loss: 0.6034 - val_iou_score: 0.3479 - val_f1-score: 0.3938
Epoch 3/3
57/57 [==============================] - 20s 355ms/step - loss: 0.2497 - iou_score: 0.6316 - f1-score: 0.7505 - val_loss: 0.6031 - val_iou_score: 0.3478 - val_f1-score: 0.3940


30


...

Epoch 1/3
67/67 [==============================] - 33s 386ms/step - loss: 0.2857 - iou_score: 0.5964 - f1-score: 0.7153 - val_loss: 0.6853 - val_iou_score: 0.3057 - val_f1-score: 0.3306
Epoch 2/3
67/67 [==============================] - 24s 363ms/step - loss: 0.2328 - iou_score: 0.6529 - f1-score: 0.7681 - val_loss: 0.6843 - val_iou_score: 0.2966 - val_f1-score: 0.3230
Epoch 3/3
67/67 [==============================] - 24s 357ms/step - loss: 0.2232 - iou_score: 0.6632 - f1-score: 0.7761 - val_loss: 0.6874 - val_iou_score: 0.2937 - val_f1-score: 0.3200


48


Epoch 1/3
38/38 [==============================] - 24s 435ms/step - loss: 0.2640 - iou_score: 0.6189 - f1-score: 0.7379 - val_loss: 0.5036 - val_iou_score: 0.4363 - val_f1-score: 0.4979
Epoch 2/3
38/38 [==============================] - 15s 390ms/step - loss: 0.2334 - iou_score: 0.6523 - f1-score: 0.7666 - val_loss: 0.4452 - val_iou_score: 0.4946 - val_f1-score: 0.5563
Epoch 3/3
38/38 [==============================] - 15s 389ms/step - loss: 0.2161 - iou_score: 0.6730 - f1-score: 0.7853 - val_loss: 0.4316 - val_iou_score: 0.5086 - val_f1-score: 0.5698


49


Epoch 1/3
72/72 [==============================] - 37s 410ms/step - loss: 0.2592 - iou_score: 0.6261 - f1-score: 0.7387 - val_loss: 0.5889 - val_iou_score: 0.3794 - val_f1-score: 0.4207
Epoch 2/3
72/72 [==============================] - 27s 379ms/step - loss: 0.2238 - iou_score: 0.6673 - f1-score: 0.7737 - val_loss: 0.5756 - val_iou_score: 0.3898 - val_f1-score: 0.4341
Epoch 3/3
72/72 [==============================] - 27s 375ms/step - loss: 0.2113 - iou_score: 0.6800 - f1-score: 0.7881 - val_loss: 0.5952 - val_iou_score: 0.3759 - val_f1-score: 0.4153


# Model

In [ ]:
# fit model
# if you use data generator use model.fit_generator(...) instead of model.fit(...)
# more about `fit_generator` here: https://keras.io/models/sequential/#fit_generator
# model.fit(
#    x=train_X,
#    y=train_y,
#    batch_size=32,
#    epochs=1,
#    validation_data=(val_X, val_y),
# )

In [ ]:
# model.summary()

In [19]:
model_name = "inceptionv3"
model_path = "/content/drive/MyDrive/Bioimaging/Bioimagining Project/UNet_model/testing_backbones"
model_path = os.path.join(model_path, model_name)
model = keras.models.load_model(model_path, compile = False)
model.compile(
  optim,
  loss=sm.losses.DiceLoss(),
  metrics=[sm.metrics.IOUScore(), sm.metrics.FScore()],
)

In [20]:
test_X, test_y = process_data_more(1, "mix", "test")
test_X, test_y = np.reshape(test_X, (-1, test_X.shape[2], test_X.shape[3], test_X.shape[4])) , np.reshape(test_y, (-1, test_y.shape[2], test_y.shape[3], test_y.shape[4]))
print(model.metrics_names)
model.evaluate(
   x=test_X,
   y=test_y,
   batch_size=32
)
test_y = np.argmax(test_y, axis = 3)

[]
10/10 [==============================] - 6s 343ms/step - loss: 0.6675 - iou_score: 0.3241 - f1-score: 0.3477


In [21]:
pred_y = model(test_X[0:150, :, :, :])
pred_y = np.argmax(pred_y, axis = 3)


In [ ]:
frame = 23
fig, ax = plt.subplots(1, 2)
im0 = ax[0].imshow(pred_y[frame, :, :])
colors = np.unique(pred_y[frame, :, :])
plt.colorbar(im0, ax = ax[0])
ax[0].set_title('Predicted')
plt.axis('off')
im1 = ax[1].imshow(test_y[frame, :, :])
colors = np.unique(test_y[frame, :, :])
plt.colorbar(im1, ax = ax[1])
ax[1].set_title('Target')
plt.axis('off')

In [ ]:
pred_y_1 = np.argmax(pred_y, axis = 3)
pred_y_1 
print(pred_y_1[100, :, :])

In [ ]:
def int_and_union(img, gt): 
    intersection = np.sum(img & gt)
    union = tf.size(img) + tf.size(gt)
    return intersection, union 

def dice_coef(intersection_arr, union_arr, smooth=0.0001):
  return 2 * (np.sum(intersection_arr) + smooth) / (np.sum(union_arr) + smooth)

In [ ]:
model_name = "inceptionv3"
model_path = "/content/drive/MyDrive/Bioimaging/Bioimagining Project/UNet_model/testing_backbones"
model_path = os.path.join(model_path, model_name)
optim = keras.optimizers.Adam(0.0001)
model = keras.models.load_model(model_path, compile = False)
model.compile(
  optim,
  loss=sm.losses.DiceLoss(),
  metrics=[sm.metrics.IOUScore(), sm.metrics.FScore()],
)
num_split = 50
for i in range(num_split): 
  test_X, test_y = process_data_more(i, "mix", "test")
  #collasing image modalities
  test_X, test_y = np.reshape(test_X, (-1, test_X.shape[2], test_X.shape[3], test_X.shape[4])) , np.reshape(test_y, (-1, test_y.shape[2], test_y.shape[3], test_y.shape[4]))
  print(test_X.shape, test_y.shape)
  # for sub in range(0, test_X.shape[0], 25): 
  #   pred_y = model(test_X[sub:max(sub+25, test_X.shape[0])])
  # pred_y = model(test_X)
  # pred_y = np.argmax(pred_y, axis = 3)
  # pred_y = np.eye(4)[pred_y]
  # test_y = tf.cast(test_y, bool)
  # pred_y = tf.cast(pred_y, bool)
  # _, _, dc = model.evaluate(
  #   x=test_X,
  #   y=test_y,
  #   batch_size=32
  # )

  # intersection, union = int_and_union(pred_y, test_y)


  dice = sm.metrics.FScore()

  dice_path = "/content/drive/MyDrive/Bioimaging/Bioimagining Project/Dice"
  batch_sz = 64
  for sub in range(0, test_X.shape[0], batch_sz):
    pred_y = model(test_X[sub:min(sub+batch_sz, test_X.shape[0])])
    pred_y = np.argmax(pred_y, axis = 3)
    pred_y = np.eye(4)[pred_y]
    bwhole_y = np.max(pred_y[:, :, :, 1:], axis = 3)
    btest_y = np.max(test_y[sub:min(sub+batch_sz, test_X.shape[0]), :, :, 1:], axis = 3)
    dc = dice(bwhole_y.astype('float32'), btest_y.astype('float32'))

    for label in range(pred_y.shape[3]): 
      ldc = dice(pred_y[:, :, :, label].astype('float32'), test_y[sub:min(sub+batch_sz, test_X.shape[0]), :, :, label].astype('float32'))
      fname = os.path.join(dice_path, f"UNET_DC_{label}_final")
      arr = np.array([])
      if os.path.exists(fname):
          with open(fname, 'rb') as f:
            arr = np.load(f)
      arr = np.append(arr, ldc)
      with open(fname, 'wb') as f:
        np.save(f, arr)

    fname = os.path.join(dice_path, "UNET_DC_AVG_final")
    arr = np.array([])
    if os.path.exists(fname):
        with open(fname, 'rb') as f:
          arr = np.load(f)
    arr = np.append(arr, dc)
    with open(fname, 'wb') as f:
      np.save(f, arr)
  
  fname = os.path.join(dice_path, "UNET_DC_AVG_final")
  arr = np.array([])
  if os.path.exists(fname):
      with open(fname, 'rb') as f:
        arr = np.load(f)
  print("Moving Dice Score at {}: {}".format(i, np.mean(arr)))

  for label in range(pred_y.shape[3]): 
    fname = os.path.join(dice_path, f"UNET_DC_{label}_final")
    arr = np.array([])
    if os.path.exists(fname):
        with open(fname, 'rb') as f:
          arr = np.load(f)
    print("Moving Dice Score for label {} at {}: {}".format(label, i, np.mean(arr)))
  
  
  # iu_list = [intersection, union]
  # names = ["UNet_intersection.npy", "UNet_union.npy"]
  # iu = [np.array([]), np.array([])]

  # for j in range(2): 
  #   var = names[j]
  #   fname = os.path.join(dice_path, var)
  #   if os.path.exists(fname):
  #     with open(fname, 'rb') as f:
  #       iu[j] = np.load(f)
  #   iu[j] = np.append(iu[j], iu_list[j])
  #   with open(fname, 'wb') as f:
  #       np.save(f, iu[j])

  #print("Moving Dice Score at {}: {}".format(i, dice_coef(iu[0], iu[1])))



(304, 256, 256, 3) (304, 256, 256, 4)
Moving Dice Score at 0: 0.543572802008028
Moving Dice Score for label 0 at 0: 0.9939386971138175
Moving Dice Score for label 1 at 0: 0.2597814745784023
Moving Dice Score for label 2 at 0: 0.3949631249350078
Moving Dice Score for label 3 at 0: 0.29360182785912625
(296, 256, 256, 3) (296, 256, 256, 4)
Moving Dice Score at 1: 0.5426445663240913
Moving Dice Score for label 0 at 1: 0.9939312772317366
Moving Dice Score for label 1 at 1: 0.2573221330635215
Moving Dice Score for label 2 at 1: 0.39287102900540366
Moving Dice Score for label 3 at 1: 0.29199439646396136
(368, 256, 256, 3) (368, 256, 256, 4)
Moving Dice Score at 2: 0.5428055287117669
Moving Dice Score for label 0 at 2: 0.9938729639803425
Moving Dice Score for label 1 at 2: 0.2576967161514175
Moving Dice Score for label 2 at 2: 0.3933559009946029
Moving Dice Score for label 3 at 2: 0.2910203726954711
(200, 256, 256, 3) (200, 256, 256, 4)
Moving Dice Score at 3: 0.5388925437191706
Moving Dice Sc